# Imports (compatible for Python 2 and Python 3)

In [58]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [59]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

In [60]:
query = '''SELECT * FROM fb55.citibike; '''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


In [61]:
#Sort data by both start station id ascending, and trip duration descending

query = '''SELECT * FROM fb55.citibike
ORDER BY start_station_name ASC, tripduration DESC'''

table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,10 Ave & W 28 St,NaN,13369,-74.001768,2062,2015-02-04 07:36:00+00,2015-02-04 08:10:00+00,489,13316,40.750664,534,Water - Whitehall Plaza,40.702551,-74.012723,16536,Subscriber,1983.0,1
1,NaN,10 Ave & W 28 St,NaN,20157,-74.001768,1984,2015-02-04 19:28:00+00,2015-02-04 20:01:00+00,489,20103,40.750664,2022,E 59 St & Sutton Pl,40.758491,-73.959206,17452,Subscriber,1985.0,1
2,NaN,10 Ave & W 28 St,NaN,11615,-74.001768,1662,2015-02-03 18:50:00+00,2015-02-03 19:18:00+00,489,11554,40.750664,309,Murray St & West St,40.714979,-74.013012,19626,Subscriber,1981.0,1
3,NaN,10 Ave & W 28 St,NaN,22495,-74.001768,1620,2015-02-05 07:31:00+00,2015-02-05 07:58:00+00,489,22447,40.750664,534,Water - Whitehall Plaza,40.702551,-74.012723,17993,Subscriber,1983.0,1
4,NaN,10 Ave & W 28 St,NaN,20713,-74.001768,1571,2015-02-04 20:25:00+00,2015-02-04 20:52:00+00,489,20657,40.750664,280,E 10 St & 5 Ave,40.733320,-73.995101,20854,Subscriber,1970.0,1


In [62]:
#Only checking trips with duration <= 3 hours

query = '''SELECT * FROM fb55.citibike
WHERE tripduration <= 180
ORDER BY start_station_name ASC, tripduration DESC'''

table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,10 Ave & W 28 St,NaN,46087,-74.001768,172,2015-02-07 23:23:00+00,2015-02-07 23:26:00+00,489,46088,40.750664,494,W 26 St & 8 Ave,40.747348,-73.997236,18068,Subscriber,1985.0,2
1,NaN,10 Ave & W 28 St,NaN,1155,-74.001768,101,2015-02-01 10:54:00+00,2015-02-01 10:55:00+00,489,1084,40.750664,489,10 Ave & W 28 St,40.750664,-74.001768,20955,Subscriber,1974.0,1
2,NaN,11 Ave & W 27 St,NaN,29475,-74.005226,167,2015-02-05 18:55:00+00,2015-02-05 18:58:00+00,458,29432,40.751396,489,10 Ave & W 28 St,40.750664,-74.001768,17092,Subscriber,1987.0,1
3,NaN,11 Ave & W 27 St,NaN,15833,-74.005226,159,2015-02-04 12:19:00+00,2015-02-04 12:22:00+00,458,15790,40.751396,489,10 Ave & W 28 St,40.750664,-74.001768,21398,Subscriber,1974.0,1
4,NaN,11 Ave & W 27 St,NaN,25929,-74.005226,112,2015-02-05 13:02:00+00,2015-02-05 13:04:00+00,458,25889,40.751396,388,W 26 St & 10 Ave,40.749718,-74.002950,15965,Subscriber,1980.0,1


In [63]:
#Select last 10 records of the table

query = '''SELECT * FROM fb55.citibike
ORDER BY cartodb_id DESC
LIMIT 10;'''

table = get_data(query)
table.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 4 St & 2 Ave,NaN,46199,-73.989780,917,2015-02-07 23:59:00+00,2015-02-08 00:15:00+00,439,46200,40.726281,417,Barclay St & Church St,40.712912,-74.010202,20998,Subscriber,1965,2
1,NaN,Carmine St & 6 Ave,NaN,46198,-74.002150,548,2015-02-07 23:58:00+00,2015-02-08 00:08:00+00,368,46199,40.730386,334,W 20 St & 7 Ave,40.742388,-73.997262,19540,Subscriber,1983,2
2,NaN,5 Ave & E 29 St,NaN,46197,-73.986831,392,2015-02-07 23:57:00+00,2015-02-08 00:03:00+00,474,46198,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,15545,Subscriber,1986,1
3,NaN,5 Ave & E 29 St,NaN,46196,-73.986831,428,2015-02-07 23:57:00+00,2015-02-08 00:04:00+00,474,46197,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,16395,Subscriber,1986,2
4,NaN,W 20 St & 8 Ave,NaN,46195,-74.000040,689,2015-02-07 23:57:00+00,2015-02-08 00:08:00+00,470,46196,40.743453,325,E 19 St & 3 Ave,40.736245,-73.984738,15585,Subscriber,1953,1
5,NaN,1 Ave & E 44 St,NaN,46194,-73.969053,1422,2015-02-07 23:57:00+00,2015-02-08 00:20:00+00,455,46195,40.750020,265,Stanton St & Chrystie St,40.722293,-73.991475,20184,Subscriber,1960,2
6,NaN,E 20 St & 2 Ave,NaN,46193,-73.982050,993,2015-02-07 23:56:00+00,2015-02-08 00:13:00+00,461,46194,40.735877,295,Pike St & E Broadway,40.714067,-73.992939,16722,Subscriber,1974,1
7,NaN,Warren St & Church St,NaN,46192,-74.009106,1165,2015-02-07 23:56:00+00,2015-02-08 00:15:00+00,152,46193,40.714740,325,E 19 St & 3 Ave,40.736245,-73.984738,16978,Subscriber,1959,1
8,NaN,W 47 St & 10 Ave,NaN,46191,-73.993012,1248,2015-02-07 23:56:00+00,2015-02-08 00:17:00+00,495,46192,40.762699,432,E 7 St & Avenue A,40.726218,-73.983799,16300,Subscriber,1984,1
9,NaN,W 17 St & 8 Ave,NaN,46190,-74.001497,306,2015-02-07 23:55:00+00,2015-02-08 00:01:00+00,116,46191,40.741776,494,W 26 St & 8 Ave,40.747348,-73.997236,14736,Subscriber,1983,2


In [64]:
#List all unique birth years

query = '''SELECT DISTINCT birth_year FROM fb55.citibike;'''

table = get_data(query)
table.head(5)

,birth_year,Unnamed: 1
0,1994.0,NaN
1,1954.0,NaN
2,1979.0,NaN
3,1956.0,NaN
4,1943.0,NaN


In [65]:
#List all unique start_station_ids

query = '''SELECT DISTINCT start_station_id FROM fb55.citibike;'''

table = get_data(query)
table.head(5)

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN


In [66]:
#Count the number of trips

query = '''SELECT count(*) FROM fb55.citibike;'''

table = get_data(query)
table.head(5)

,count,Unnamed: 1
0,46200,NaN


In [67]:
#FInd minimal, maximal and average trip duration

query = '''SELECT MAX(tripduration), MIN(tripduration), AVG(tripduration) from fb55.citibike;'''

table = get_data(query)
table.head(5)

,max,min,avg
0,43016,60,675.865823


In [68]:
#Select only trips that started at 1 AM

query = '''SELECT starttime FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) IN (1);'''

table = get_data(query)
table.head(5)

,starttime,Unnamed: 1
0,2015-02-01 01:23:00+00,NaN
1,2015-02-01 01:11:00+00,NaN
2,2015-02-06 01:13:00+00,NaN
3,2015-02-01 01:03:00+00,NaN
4,2015-02-01 01:04:00+00,NaN


In [69]:
#What is the average birth year of people that ride bikes at 2 AM?

query = '''SELECT AVG(birth_year) from fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) IN (2);'''

table = get_data(query)
table.head(5)

,avg,Unnamed: 1
0,1978.201754,NaN


In [70]:
#What is the age of the oldest person riding at 3AM?

query = '''SELECT MAX(2018 - birth_year) from fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) IN (3)'''

table = get_data(query)
table.head(5)

,max,Unnamed: 1
0,78,NaN


In [71]:
#Selecting trips started on Feb-02-2015 only

query = '''SELECT * from fb55.citibike
WHERE starttime BETWEEN '02/02/2015 00:00:00.00' AND '02/02/2015 23:59:59.999'
'''

table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


In [72]:
#Selecting trips started on the weekends 

query = '''SELECT * from fb55.citibike
WHERE EXTRACT(DOW FROM starttime::date) IN (0,6)
'''

table = get_data(query)
table.head(20)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0
5,NaN,E 20 St & 2 Ave,NaN,39894,-73.982050,296,2015-02-07 08:13:00+00,2015-02-07 08:18:00+00,461,39876,40.735877,435,W 21 St & 6 Ave,40.741740,-73.994156,17278,Subscriber,1987.0,1
6,NaN,Broadway & W 60 St,NaN,41512,-73.981918,600,2015-02-07 12:35:00+00,2015-02-07 12:45:00+00,499,41501,40.769155,523,W 38 St & 8 Ave,40.754666,-73.991382,15224,Subscriber,1959.0,1
7,NaN,E 6 St & Avenue D,NaN,41595,-73.976687,1203,2015-02-07 12:47:00+00,2015-02-07 13:07:00+00,411,41585,40.722281,428,E 3 St & 1 Ave,40.724677,-73.987834,21461,Subscriber,1968.0,1
8,NaN,Great Jones St,NaN,41806,-73.993790,520,2015-02-07 13:16:00+00,2015-02-07 13:25:00+00,229,41796,40.727434,168,W 18 St & 6 Ave,40.739713,-73.994564,14621,Subscriber,1964.0,1
9,NaN,W 13 St & 6 Ave,NaN,42316,-73.997044,487,2015-02-07 14:18:00+00,2015-02-07 14:26:00+00,345,42307,40.736494,509,9 Ave & W 22 St,40.745497,-74.001971,21362,Subscriber,1962.0,1


In [73]:
#Weekend: select average trip duration

query = '''SELECT AVG(tripduration) from fb55.citibike
WHERE EXTRACT(DOW FROM starttime::date) IN (0,6)
'''

table = get_data(query)
table.head(20)

,avg,Unnamed: 1
0,662.942181,NaN


In [74]:
#Selecting weekday trips

query = '''SELECT starttime from fb55.citibike
WHERE EXTRACT(DOW FROM starttime::date) IN (1,2,3,4,5)
'''

table = get_data(query)
table.head(20)

,starttime,Unnamed: 1
0,2015-02-02 17:44:00+00,NaN
1,2015-02-02 20:12:00+00,NaN
2,2015-02-03 08:02:00+00,NaN
3,2015-02-03 12:31:00+00,NaN
4,2015-02-03 16:45:00+00,NaN
5,2015-02-03 17:37:00+00,NaN
6,2015-02-03 18:21:00+00,NaN
7,2015-02-03 20:26:00+00,NaN
8,2015-02-03 20:37:00+00,NaN
9,2015-02-03 22:20:00+00,NaN


In [75]:
#Group start station names, showing the list of start station locations, showing the number of trips started per station 

query = '''SELECT start_station_name, COUNT(start_station_name) FROM fb55.citibike
GROUP BY start_station_name'''

table = get_data(query)
table.head(5)

,start_station_name,count
0,E 30 St & Park Ave S,227
1,Pearl St & Anchorage Pl,54
2,W 56 St & 6 Ave,164
3,West Thames St,107
4,Bank St & Washington St,125


In [76]:
#but only for stations within 500m of Time Square! The coordinates of Time Square is (40.7577,-73.9857)
#In this case, average length of degree between latitude and longitude is 110938 meters

query = '''SELECT start_station_name, COUNT(start_station_name), (SQRT(POWER(start_station_latitude - 40.7577,2) + POWER(start_station_longitude + 73.9857, 2)) * 110938) AS DIST
FROM fb55.citibike
WHERE (SQRT(POWER(start_station_latitude - 40.7577,2) + POWER(start_station_longitude + 73.9857, 2)) * 110938) < 502
GROUP BY start_station_name, start_station_latitude, start_station_longitude
'''

table = get_data(query)
table.head(5)

,start_station_name,count,dist
0,W 43 St & 6 Ave,112,388.981725
1,Broadway & W 41 St,251,300.788590
2,W 45 St & 6 Ave,141,325.057521
3,Broadway & W 49 St,213,356.129026
4,W 45 St & 8 Ave,141,366.629780


In [77]:
#Find the start station id that had the highest number of bikes taken from it

query = '''SELECT start_station_id, COUNT(bikeid) as bike FROM fb55.citibike
GROUP BY start_station_id
ORDER BY bike DESC
LIMIT 1;'''

table = get_data(query)
table.head(5)

,start_station_id,bike
0,521,530


In [78]:
#Show top 3 end station id with the largest total trip duration

query = '''SELECT end_station_id, SUM(tripduration) FROM fb55.citibike
GROUP BY end_station_id
LIMIT 3;'''

table = get_data(query)
table.head(5)

,end_station_id,sum
0,120,15605
1,285,203039
2,251,132752


In [79]:
#Find the start station id with the shortest average trip duration during 1 AM.

query = '''SELECT start_station_id, AVG(tripduration) FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) IN (1)
GROUP BY start_station_id
ORDER BY avg ASC
LIMIT 1;'''

table = get_data(query)
table.head(5)

,start_station_id,avg
0,316,77.0


In [80]:
#Find the station that had the longest average trip duration during #weekends and within 500m of TimeSquare!

query = '''SELECT start_station_name, AVG(tripduration), (SQRT(POWER(start_station_latitude - 40.7577,2) + POWER(start_station_longitude + 73.9857, 2)) * 110938) AS DIST
FROM fb55.citibike
WHERE (SQRT(POWER(start_station_latitude - 40.7577,2) + POWER(start_station_longitude + 73.9857, 2)) * 110938) < 502 AND EXTRACT(DOW FROM starttime::date) IN (0,6)
GROUP BY start_station_name, start_station_latitude, start_station_longitude, starttime, tripduration
ORDER BY AVG(tripduration) DESC
LIMIT 1;
'''

table = get_data(query)
table.head(5)


,start_station_name,avg,dist
0,Broadway & W 49 St,15636.0,356.129026
